
MCM 2021
Murder Hornets

In [39]:
import numpy as np
import pandas as pd
import nltk

In [37]:
df = pd.read_csv('/Users/allisonliu/Desktop/MCM2021/data/2021_MCM_Problem_C_Data/2021MCMProblemC_DataSet.csv')
df_images = pd.read_csv('/Users/allisonliu/Desktop/MCM2021/data/2021_MCM_Problem_C_Data/2021MCM_ProblemC_ Images_by_GlobalID.csv')
    
df_positive = df.loc[df['Lab Status'] == 'Positive ID']
df_negative = df.loc[df['Lab Status'] == 'Negative ID']
df_unprocessed = df.loc[df['Lab Status'] == 'Unprocessed']

num_pos = len(df_positive)
num_neg = len(df_negative)
num_unproc = len(df_unprocessed)